In [2]:
import os 
root_dir = os.getcwd()
artifacts_dir = os.path.join(root_dir, "artifacts")
data_dir = os.path.join(artifacts_dir, "data_dir")

X = os.path.join(data_dir, "X.pickle")
y = os.path.join(data_dir, "y.pickle")

In [3]:
challenge_dir = os.path.join(root_dir, "Multilabel")
img_dir = os.path.join(challenge_dir, "photos")
print(os.path.exists(img_dir))

True


In [4]:
#from keras.applications.vgg16 import decode_predictions
from keras.applications.regnet import preprocess_input
import tensorflow as tf
from keras.utils import load_img
import matplotlib.pyplot as plt 
from PIL import Image 
import seaborn as sns
import pandas as pd 
import numpy as np 
import os 
from tensorflow.keras.models import Sequential
#tf.keras.utils.img_to_array 
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.layers import Dense
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras.models import Model
tf.data.experimental.enable_debug_mode()

In [5]:
#X = pickle.load(open(X, "rb"))
import pickle
y = pickle.load(open(y, "rb"))

NameError: name 'df' is not defined

In [6]:
raw_data = []

def _load_image(img_dir):
    for num, img in enumerate(os.listdir(img_dir)):
        img_path = os.path.join(img_dir, img)
        im = load_img(img_path, target_size = (224, 224,3))
        im = img_to_array(im)
        #im  = np.expand_dims(im, axis =0)
        #print(im.shape)
        im = preprocess_input(im)
        #print(im.shape)
        raw_data.append([im, [y[num][0], y[num][1], y[num][2],y[num][3]]])


_load_image(img_dir)



In [7]:
X = []
y = []
for i,data in enumerate(raw_data):
    X.append(raw_data[i][0])
    y.append(raw_data[i][1])

In [8]:
y = np.array(y)
X = np.array(X)

In [9]:
a,b,c, d = [], [] , [], []
for i in y:
    a.append(i[0]), b.append(i[1]), c.append(i[2]), d.append(i[3])

In [10]:
a = np.array(a)
b = np.array(b)
c = np.array(c)
d = np.array(d)

In [11]:
model1 = Sequential()

# 1st layer as the lumpsum weights from resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5
# NOTE that this layer will be set below as NOT TRAINABLE, i.e., use it as is
model1.add( tf.keras.applications.ResNet50(include_top = False, pooling = 'avg', weights = 'imagenet', input_shape=(224, 224, 3)))

# 2nd layer as Dense for 2-class classification, i.e., dog or cat using SoftMax activation
model1.add(Dense(1, activation = 'sigmoid'))

# Say not to train first layer (ResNet) model as it is already trained
model1.layers[0].trainable = False

model1.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), optimizer='adam', metrics=tf.keras.metrics.BinaryAccuracy())

In [36]:
from keras import layers
from keras import Input
resnet_base = tf.keras.applications.ResNet50(include_top = False, pooling = 'avg', weights = 'imagenet')
for layer in resnet_base.layers:
    layer.trainable = False
xinput = Input(shape=(224, 224, 3))
inpu = resnet_base(xinput)


prediction1 = layers.Dense(1, activation='sigmoid', name='a')(inpu)
prediction2 = layers.Dense(1, activation='sigmoid', name='b')(inpu)
prediction3 = layers.Dense(1, activation='sigmoid', name='c')(inpu)
prediction4 = layers.Dense(1, activation='sigmoid', name='d')(inpu)





In [13]:
model = Model(xinput,[prediction1, prediction2, prediction3, prediction4])

In [14]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 resnet50 (Functional)          (None, 2048)         23587712    ['input_3[0][0]']                
                                                                                                  
 a (Dense)                      (None, 1)            2049        ['resnet50[0][0]']               
                                                                                                  
 b (Dense)                      (None, 1)            2049        ['resnet50[0][0]']           

In [15]:
model.compile(loss = ["binary_crossentropy", "binary_crossentropy","binary_crossentropy","binary_crossentropy"], optimizer="adam")

In [20]:
model.fit(X, [a,b,c,d], epochs=2)

Epoch 1/2
31/31 [==============================] - 56s 2s/step - loss: 3.6359e-04 - a_loss: 8.4373e-05 - b_loss: 9.4289e-05 - c_loss: 1.0519e-04 - d_loss: 7.9740e-05
Epoch 2/2
31/31 [==============================] - 58s 2s/step - loss: 3.0398e-04 - a_loss: 7.1624e-05 - b_loss: 7.9338e-05 - c_loss: 8.8107e-05 - d_loss: 6.4908e-05


In [17]:
im = load_img(r"Multilabel\photos\image_005.jpg", target_size = (224, 224,3))
im = img_to_array(im)
        #im  = np.expand_dims(im, axis =0)
        #print(im.shape)
im = preprocess_input(im)

In [18]:
im = im.reshape(1, im.shape[0], im.shape[1],3)

In [19]:
model.predict(im)

1/1 [==============================] - 1s 888ms/step


[array([[0.9994757]], dtype=float32),
 array([[0.999961]], dtype=float32),
 array([[7.905688e-05]], dtype=float32),
 array([[0.00011136]], dtype=float32)]

In [23]:

root_dir = os.getcwd()
artifacts_dir = os.path.join(root_dir, "artifacts")
prepared_data_dir = os.path.join(artifacts_dir , "prepared")
clenaed_csv  = os.path.join(prepared_data_dir, "cleaned.csv")

raw_data_dir = os.path.join(root_dir, "Multilabel")
raw_label_file = os.path.join(raw_data_dir, "labels.txt")
raw_img_dir = os.path.join(raw_data_dir, "photos")


In [25]:
df = pd.read_csv(clenaed_csv)

In [32]:
df.head()

,a,b,c,d
0,1.0,1.0,0.0,1.0
1,1.0,0.0,0.0,0.0
2,1.0,1.0,0.0,0.0
3,1.0,1.0,0.0,0.0
4,1.0,1.0,0.0,0.0
